# The Countdown Show, Numbers-Game 
Countdown is a british show that broadcast on Channel 4. In the show, there are three games that are played in the show. We'll be concerned with the game that is called "Countdown" - Essentially, the contestants must use arithmetic mathematics to reach a random target number from six other numbers. The playing numbers as well as the target number are randomly generated, and you need to reach the target number, using the playing numbers and performing the arithmetic operations on these numbers.

We can use the reverse polish notation in finding of the target number using all the possible permutations (combinations) of the numbers and the operators.

## Complexity of the Problem
The problem is fairly complex and has a lot of possible solutions. In some scenarios, there can be a lot of solutions, and in others, there's a possibility that the number won't be able to be reached, in which scenario the closer player will be the winner.

You get 6 playing numbers to perform arithmetics drawn from a pile of big numbers, as well small numbers, the possible permutations (combinations) of the numbers are 720 - With all of the possibilities of 5 different operators, the possibilities shoot up to 737280 various combinations (but not limited to).

Brute-forcing the solution might not be the most efficient solution. 


In [320]:
# Import all the libraries that the lab used.
import itertools as it
import random
import operator
import pandas as pd

# Generate the list of the "large" numbers.
def generate_large(step = 25, amount = 4):
	res = []

	for i in range(1, amount + 1):
		res.append( i * step )

	return res

# Generating the small list, we'll use the same function we used to get the large numbers, except we'll manipulate (order and double the amount of them, not the value) the numbers
def generate_small():
	return sorted( generate_large(step = 1, amount = 10) * 2 )

def pick_random(list, amount = 1):
	return random.sample(list, amount)

def permutate(list, amount = 2):
	return it.permutations(list, amount)

def permutate_limited(list, amount, limit):
	return it.islice(permutate(list, amount), limit)

def numbers_game(large = None):
	large = large if large else random.randrange(0, 5)

	random_large = pick_random(generate_large(amount=large), amount = large)
	random_small = pick_random(generate_small(), amount = 6 - large)

	play_numbers = random_large + random_small

	target_number = random.randrange(101, 1000)

	return play_numbers, target_number



In [321]:
# Generate large numbers
large_numbers = generate_large(25, 4)
# Generate small numbers
small_numbers = generate_small()

# Fit that data into a pandas data frame
df = pd.DataFrame(large_numbers, columns = ['Large'])
# And finally display it
df

,Large
0,25
1,50
2,75
3,100


In [322]:
# Peek the small numbers as well, just to make sure we're correct. We'll peek only 5 of them, to keep this preview short, and to keep the notebook clean.
# Trust me, there's 20 elements in the list.
df = pd.DataFrame(small_numbers[:5], columns = ['Small'], )
df

,Small
0,1
1,1
2,2
3,2
4,3


In [323]:
# Or if you don't trust me, then see for yourself.
len(small_numbers)

20

In [324]:
play_numbers, target = numbers_game()

play_numbers, target

([75, 50, 25, 100, 7, 2], 555)

In [325]:
OP = [operator.add, operator.mul, operator.sub, operator.truediv]

# simplify the target for demo purposes, it'll be achievable using two numbers.
temp_target = max(play_numbers) * min(play_numbers);

# Will return all of the combinations of numbers and arithmetic operators that achieve the target number - We can adjust the amount of numbers we want to use to generate permutations
def find_hits(play_numbers, target):
	return list(filter(lambda z: z[1](z[0][0], z[0][1]) == target, it.product(permutate(play_numbers, 2), OP)));

# Problem is, we're using temporary target which has been made easier on purpose for the algorithm to find answer using any two numbers and an operator - We'll need
# something a little more bit advanced if we want to find the answer using more than two numbers, and any amount of arithmetic operators.
print("Simplified target: %s, real target: %s" % (temp_target, target))

find_hits(play_numbers, temp_target)

# We'll cover the solution to this problem in the next part, using the reverse polish notation.
# By the way, reverse polish notation is a way to write arithmetic expressions in a way that operators are applied to the numbers in the order they appear in the expression.
# It'll help us in parsing of the notation programmatically.
# The reverse polish scheme was actually reinvented several times, with Edsger Dijkstra's being one of the reinventors in the early 60s in order to simplify the notation for the easier computation of it by 
# the computer, as well as to reduce computer memory usage.
# The reverse polish notation is also known as postfix notation - It's the notation used in various stack-oriented programming languages. It has lead to faster computations and less human-errors caused by humans using
# the notation, implying that the notation might be easier to understand and read. (Honestly, how many times have you mistaken order of operations due to brackets?)
# ^ Needs citation, but I'll leave it for now. Some evidence suggests that it's harder to learn, and that less errors resulting from it are due to less keystrokes needed to type it out.

# An example notation would look like: 3 4 + 5 *
# It's 35, by the way.

Simplified target: 200, real target: 555


[((100, 2), <function _operator.mul(a, b, /)>),
 ((2, 100), <function _operator.mul(a, b, /)>)]

In [326]:
# Let's generate a postfix notation list of combinations.
# This is a generator - note the yield statements, this is a feature of generators in Python which facilitates the functional programming aspect of Python.
# Those values will be generated on the fly, and will not be stored in the memory - Each invocation of the generator will return a new value.

# Generators let us not pre-compute values, but rather generate them on the fly - We can then simply abort (return) from within the for block and the generator won't compute any further values.
def patterns(numbers, operators):
	# We're recursing, so we'll need to make sure we don't go too deep.
	if len(numbers) == 1:
		yield numbers

	for i in range(1, len(numbers)):
		for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
			yield [*left, *right, operators[0]]


# Function that can evaluate the RPN notation expression.
def eval_rpn(rpn):
	stack = []
	# For each RPN element, we'll check if it's an operator or a number.
	for i in rpn:
		if isinstance(i, int):
			# If it's a number, we'll push it to the stack.
			stack = stack + [i]
	else:
		# If it's an operator, we'll pop the last two elements from the stack, and apply the operator to the two numbers.
		right = stack[-1]
		stack = stack[:-1]
		left = stack[-1]
		stack = stack[:-1]
		stack = stack + [i(left, right)]

	# We'll return the last element of the stack, which should be the result of the expression.
	return stack[0]

# We would probably generate all the permutations of the numbers and operators, then possible permutations of the RPN operations with those numbers and then evaluate those notations and filter out
# the ones that hit the target and save those as solutions. However, it's worth nothing that it is a ridiculous amount of possibilities to consider and bruteforce in order to find the answer.

In [327]:
# Search for the target number solution using RPN notation and permutations of numbers and operators.
def search_rpn(numbers, operators, target):
	print("Target is %s" % target)

	for i in patterns(numbers, operators):
		print(i)
		result = eval_rpn(i);
		print("Result: %s" % result)

		if result == target:
			return i

solution = search_rpn(play_numbers, [operator.add, operator.add, operator.mul, operator.truediv, operator.sub], target)

Target is 555
[75, 50, 25, 100, 7, 2, <built-in function sub>, <built-in function truediv>, <built-in function mul>, <built-in function add>, <built-in function add>]
Result: 75
[75, 50, 25, 100, 7, <built-in function sub>, 2, <built-in function truediv>, <built-in function mul>, <built-in function add>, <built-in function add>]
Result: 75
[75, 50, 25, 100, <built-in function truediv>, 7, 2, <built-in function sub>, <built-in function mul>, <built-in function add>, <built-in function add>]
Result: 75
[75, 50, 25, 100, 7, <built-in function sub>, <built-in function truediv>, 2, <built-in function mul>, <built-in function add>, <built-in function add>]
Result: 75
[75, 50, 25, 100, <built-in function sub>, 7, <built-in function truediv>, 2, <built-in function mul>, <built-in function add>, <built-in function add>]
Result: 75
[75, 50, 25, <built-in function mul>, 100, 7, 2, <built-in function sub>, <built-in function truediv>, <built-in function add>, <built-in function add>]
Result: 75
[7